In [27]:
import json
from pathlib import Path
from typing import Dict, Any, List, Optional, Set

import typer
import pandas as pd


Message = {}
Context = []



def prepare_messages(tg_history_path, output_path,):
    with open(tg_history_path) as messages_file:
        messages = json.load(messages_file)['messages']

    contexts = _create_contexts(messages)
    contexts = _transform_contexts(contexts)

    contexts_df = pd.DataFrame.from_records(contexts)
    contexts_df.drop_duplicates(inplace=True)
    contexts_df.to_csv(output_path, index=False)


def _create_contexts(messages):
    replies_threads = {}
    id_to_message = {}
    for message in messages:
        id_to_message[message['id']] = message
        if 'reply_to_message_id' in message:
            replies_threads[message['reply_to_message_id']] = message['id']

    contexts = []
    cur_context = _create_default_list()
    visited_replies = set()

    for message in messages:
        if (
            message['type'] != 'message' or
            not message['text'] or
            not isinstance(message['text'], str) or
            message['id'] in visited_replies
        ):
            continue

        if 'forwarded_from' in message and cur_context:
            contexts.append(cur_context)
            cur_context = _create_default_list()
            continue

        if message['id'] in replies_threads:
            contexts.append(cur_context)
            cur_context = _create_default_list()
            _resolve_thread(contexts, replies_threads, visited_replies, id_to_message, message)
            continue

        if cur_context[-1] and message['from_id'] == cur_context[-1]['from_id']:
            contexts[-1][-1]['text'] += '\n' + message["text"]
            continue

        cur_context.pop(0)
        cur_context.append(message)
        contexts.append(cur_context.copy())

    return contexts


def _resolve_thread(
    contexts,
    replies_threads,
    visited_replies,
    id_to_message,
    message,
) -> None:
    cur_context = _create_default_list()
    cur_id = message['id']

    while cur_id:
        cur_context.pop(0)
        cur_context.append(id_to_message[cur_id])
        contexts.append(cur_context.copy())

        visited_replies.add(cur_id)
        cur_id = replies_threads.get(cur_id)


def _transform_contexts(contexts):
    return [_transform_context(context) for context in contexts if any(context)]


def _transform_context(context):
    return {
        'context_3': _transform_message(context[0]),
        'context_2': _transform_message(context[1]),
        'context_1': _transform_message(context[2]),
        'response': _transform_message(context[3]),
    }


def _transform_message(message):
    if not message:
        return None

    if isinstance(message['text'], list):
        texts = [text['text'] if isinstance(text, dict) else text for text in message['text']]
        message['text'] = ''.join(texts)

    return message['text']


def _create_default_list(message = ''):
    return [None, None, None, message]




In [163]:
with open(tg_history_path) as messages_file:
        messages = json.load(messages_file)['messages']

In [164]:
bool('s')

True

In [165]:
messages[7]

{'id': 115,
 'type': 'message',
 'date': '2023-03-01T12:44:03',
 'date_unixtime': '1677663843',
 'from': 'Аня ❤️',
 'from_id': 'user348973081',
 'forwarded_from': 'Аня ❤️',
 'text': '5. Какой наиболее часто мутирующий ген при раке? Какая у него функция?\n*',
 'text_entities': [{'type': 'plain',
   'text': '5. Какой наиболее часто мутирующий ген при раке? Какая у него функция?\n*'}]}

In [92]:
messages[-95]

{'id': 17273,
 'type': 'message',
 'date': '2023-08-24T08:05:19',
 'date_unixtime': '1692853519',
 'from': 'Мама',
 'from_id': 'user5070028513',
 'text': 'Я блины с мясом и кофе с молоком беру',
 'text_entities': [{'type': 'plain',
   'text': 'Я блины с мясом и кофе с молоком беру'}]}

In [87]:
messages[-92]

{'id': 17281,
 'type': 'message',
 'date': '2023-08-24T11:47:08',
 'date_unixtime': '1692866828',
 'from': 'No',
 'from_id': 'user5982387868',
 'text': 'Мы прилетели',
 'text_entities': [{'type': 'plain', 'text': 'Мы прилетели'}]}

In [86]:
messages[-93]

{'id': 17275,
 'type': 'message',
 'date': '2023-08-24T08:05:27',
 'date_unixtime': '1692853527',
 'from': 'No',
 'from_id': 'user5982387868',
 'reply_to_message_id': 17273,
 'text': 'Ну или так',
 'text_entities': [{'type': 'plain', 'text': 'Ну или так'}]}

In [64]:
messages[-100]

{'id': 17239,
 'type': 'message',
 'date': '2023-08-24T07:12:35',
 'date_unixtime': '1692850355',
 'from': 'Мама',
 'from_id': 'user5070028513',
 'file': '(File not included. Change data exporting settings to download.)',
 'thumbnail': '(File not included. Change data exporting settings to download.)',
 'media_type': 'sticker',
 'sticker_emoji': '😘',
 'width': 512,
 'height': 512,
 'text': '',
 'text_entities': []}

In [175]:
messages[83]

{'id': 194,
 'type': 'message',
 'date': '2023-03-13T13:12:06',
 'date_unixtime': '1678702326',
 'from': 'No',
 'from_id': 'user5982387868',
 'text': 'на 6 точно напишу',
 'text_entities': [{'type': 'plain', 'text': 'на 6 точно напишу'}]}

In [76]:
import re 

st = '''['Watch "Как отсрочить старость? Рассказывает эндокринолог" on YouTube\n', {'type': 'link', 'text': 'https://youtu.be/imgZSjmO4BY'}, '']'''
print(re.search("(?P<url>https?://[^\s]+)", st))

<re.Match object; span=(100, 131), match="https://youtu.be/imgZSjmO4BY'},">


In [114]:
len('''нтерес к машинному обучению у меня появился, когда я узнала про NLP. 
Сначала я училась на экономике и выбрала в качестве майнора ИАД, но мне хотелось больше погрузиться в Computer Science, поэтому я перепоступила на ПМИ в конце 1 курса. В марте я была на Школе искусственного Интеллекта МТС. Там я посетила лекцию Сергея Загоруйко, где он рассказывал о своей карьере и задачах, которые он решал: 3D реконструкция, детекция объектов, беспилотные автомобили; а также о том, как развивались эти области последние 10 лет. Его рассказ очень меня впечатлил. Мне нравится, что в машинном обучении есть много интересных направлений,которые позволяют решать большое количество задач на стыке разных областей. Мне было бы интересно в будущем поработать в нескольких сферах: обработке естественного языка, биоинформатике, чем-то, связанном с компьютерным зрением. На МОПе есть курсы по выбору по всем этим темам, где я смогу получше узнать каждую из них и понять, что мне ближе
Аня ❤️:переписала кусок мотивашки посмотри пожалуйста🥺🥺🥺''')

1024

In [1]:
# from datetime import datetime, timedelta
# import json

# def process_chat(chat_data):
#     # Сортировка сообщений по дате
#     sorted_chat = sorted(chat_data, key=lambda x: x['date_unixtime'])

#     dialogs = []
#     current_dialog = ['']
#     prev_time = None

#     for message in sorted_chat:
#         if message['from'] == 'No':
#             # Преобразование даты из строки в объект datetime
#             message_time = datetime.strptime(message['date'], '%Y-%m-%dT%H:%M:%S')

#             if prev_time is None or (message_time - prev_time) <= timedelta(hours=2):
#                 current_dialog.append(message['text'])
#             else:
#                 if current_dialog:
#                     dialogs.append(' '.join(current_dialog))
#                 current_dialog = [message['text']]

#             prev_time = message_time

#     if current_dialog:
#         dialogs.append(' '.join(current_dialog))

#     return dialogs

# # Пример входных данных
# input_data = [
#     # ... (ваш входной json здесь)
# ]

# dialogs = process_chat(messages)

# for i, dialog in enumerate(dialogs, start=1):
#     print(f"Dialog {i}: {dialog}\n")


In [212]:
messages

[{'id': 101,
  'type': 'message',
  'date': '2023-02-16T18:32:17',
  'date_unixtime': '1676561537',
  'from': 'Аня ❤️',
  'from_id': 'user348973081',
  'text': 'Никит?',
  'text_entities': [{'type': 'plain', 'text': 'Никит?'}]},
 {'id': 102,
  'type': 'message',
  'date': '2023-02-20T22:12:28',
  'date_unixtime': '1676920348',
  'from': 'No',
  'from_id': 'user5982387868',
  'forwarded_from': 'Other',
  'text': 'Почему я хочу попасть на школу по ИИ от МТС?\n\nИИ я начал увлекаться еще в школе, тогда я смотрел популярные лекции малого ШАДа и выступления Андрея Сербанта, потом я решил изучать программирование на си, в вузе освоил питон, прошел курсы по теорверу и матстату от CSC, в сентябре начал заниматься продуктовой аналитикой, прошел курс от Тинькофф, но потом понял, что на продуктовой аналитике я останавливаться не хочу, мне захотелось двигаться в более техническую сферу и для меня очевидным выбор был DS, я прошел отбор на курс Тинькофф по ML, параллельно изучая курс по ML от  Радос

In [53]:
import json
from pathlib import Path


from datetime import datetime

def get_dialogues(dump_path, MAX_CONTEXT_LEN = 1200, MAX_TEXT_LEN = 1200, MY_NAME = 'No'):

    with open(dump_path) as messages_file:
            messages = json.load(messages_file)['messages']

            
    dialogues = []
    last_post_date = None
    last_sender = None
    
    curr_input = ''
    curr_output = ''
    curr_context = ''
    
    print(len(messages))

    for message in messages:
        #remove forwarded messages
        if message.get('forwarded_from', None):
            continue
        
        if message.get('text', None):
            txt = message.get('text', None)
            if len(txt) > MAX_TEXT_LEN:
                continue 
            if re.search("(?P<url>https?://[^\s]+)", str(txt)): # remove links 
                continue   
        elif message.get('sticker_emoji', None):
            txt = message.get('sticker_emoji', None)
        else:
            continue
            
        curr_date = datetime.strptime(message.get('date', None),'%Y-%m-%dT%H:%M:%S') 
        curr_sender = message.get('from', None)
        
        if curr_date is None or curr_sender is None:
            continue 
        
        if last_post_date == None:
            last_post_date = curr_date
        if last_sender == None:
            last_sender = curr_sender
        
        if curr_sender == MY_NAME:
            curr_output = curr_output + str(txt) + ' \n '
        else:
            curr_input = curr_input + str(txt) + ' \n '
                
                
        if curr_sender != last_sender:
            dialogues.append({'context':curr_context, 'input':curr_input, 'output':curr_output})
            if len(curr_input) != 0:
                curr_context += 'АНЯ:' + curr_input
            if len(curr_output) != 0:
                curr_context += 'НИКИТА:' + curr_output
            curr_input = ''
            curr_output = ''
            

        if len(curr_context) > MAX_CONTEXT_LEN:
            curr_context = ''
        
        if (curr_date - last_post_date).seconds//3600 > 2: 
            
            last_post_date = None
            last_sender = None
            
            curr_input = ''
            curr_output = ''
            curr_context = ''  
        
        last_sender = curr_sender


        # TO-DO: catch reply_to_message_id
        
    return dialogues

In [35]:
check_message({'id': 111,
  'type': 'message',
  'date': '2023-02-27T22:33:22',
  'date_unixtime': '1677526402',
  'from': 'Аня ❤️',
  'from_id': 'user348973081',
  'text': [{'type': 'link',
    'text': 'https://docviewer.yandex.ru/view/1049160442/?page=3&*=3Z%2BxL2ib57dQlFuYyC2fnhrzuXh7InVybCI6InlhLWRpc2stcHVibGljOi8vNm5TbGZhdDJuaHlsUko2UDU5M01SczRBK0tuaWZaOUNVRDJLSTM0NnUzdXFCbXZjazJ0OGNXYytzZGU4dnBWb3EvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovbGVjdHVyZV9ub3Rlcy9sZWN0dXJlNV9mbWF0Y29tcDIzLnBkZiIsInRpdGxlIjoibGVjdHVyZTVfZm1hdGNvbXAyMy5wZGYiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6IjEwNDkxNjA0NDIiLCJ0cyI6MTY3NzUyNjEyMTEwMywieXUiOiI0OTkxMzczOTkxNjU5MDMwMDAzIn0%3D'}],
  'text_entities': [{'type': 'link',
    'text': 'https://docviewer.yandex.ru/view/1049160442/?page=3&*=3Z%2BxL2ib57dQlFuYyC2fnhrzuXh7InVybCI6InlhLWRpc2stcHVibGljOi8vNm5TbGZhdDJuaHlsUko2UDU5M01SczRBK0tuaWZaOUNVRDJLSTM0NnUzdXFCbXZjazJ0OGNXYytzZGU4dnBWb3EvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovbGVjdHVyZV9ub3Rlcy9sZWN0dXJlNV9mbWF0Y29tcDIzLnBkZiIsInRpdGxlIjoibGVjdHVyZTVfZm1hdGNvbXAyMy5wZGYiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6IjEwNDkxNjA0NDIiLCJ0cyI6MTY3NzUyNjEyMTEwMywieXUiOiI0OTkxMzczOTkxNjU5MDMwMDAzIn0%3D'}]}, 1000)

False

In [241]:
re.search("(?P<url>https?://[^\s]+)", '''[{'type': 'link',
   'text': 'https://docviewer.yandex.ru/view/1049160442/?page=3&*=3Z%2BxL2ib57dQlFuYyC2fnhrzuXh7InVybCI6InlhLWRpc2stcHVibGljOi8vNm5TbGZhdDJuaHlsUko2UDU5M01SczRBK0tuaWZaOUNVRDJLSTM0NnUzdXFCbXZjazJ0OGNXYytzZGU4dnBWb3EvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTovbGVjdHVyZV9ub3Rlcy9sZWN0dXJlNV9mbWF0Y29tcDIzLnBkZiIsInRpdGxlIjoibGVjdHVyZTVfZm1hdGNvbXAyMy5wZGYiLCJub2lmcmFtZSI6ZmFsc2UsInVpZCI6IjEwNDkxNjA0NDIiLCJ0cyI6MTY3NzUyNjEyMTEwMywieXUiOiI0OTkxMzczOTkxNjU5MDMwMDAzIn0%3D'}]''')

<re.Match object; span=(30, 479), match="https://docviewer.yandex.ru/view/1049160442/?page>

In [275]:
s= '''8 15\n\\             /\n.|           |.\n.|           |.\n..\\         /..\n...|       |...\n...|       |...\n....\\     /....\n.....\\___/.....\n2\ngin 2 %\ntonic 4 * \n 8 15\n\\             /\n.|           |.\n.|           |.\n..\\         /..\n...|       |...\n...|       |...\n....\\     /....\n.....\\___/.....\n2\ngin 2 %\ntonic 4 * \n \\             /\n.|***********|.\n.|***********|.\n..\\*********/..\n...|*******|...\n...|%%%%%%%|...\n....\\%%%%%/....\n.....\\___/..... \n \\             /\n.|***********|.\n.|***********|.\n..\\*********/..\n...|*******|...\n...|%%%%%%%|...\n....\\%%%%%/....\n.....\\___/..... \n '},
 {'context': 'АНЯ:😘 \n Оке приду через 5 минут \n НИКИТА:8 15\n\\             /\n.|           |.\n.|           |.\n..\\         /..\n...|       |...\n...|       |...\n....\\     /....\n.....\\___/.....\n2\ngin 2 %\ntonic 4 * \n 8 15\n\\             /\n.|           |.\n.|           |.\n..\\         /..\n...|       |...\n...|       |...\n....\\     /....\n.....\\___/.....\n2\ngin 2 %\ntonic 4 * \n \\             /\n.|***********|.\n.|***********|.\n..\\*********/..\n...|*******|...\n...|%%%%%%%|...\n....\\%%%%%/....\n.....\\___/..... \n \\             /\n.|***********|.\n.|***********|.\n..\\*********/..\n...|*******|...\n...|%%%%%%%|...\n....\\%%%%%/....\n.....\\___/..... \n '''

In [277]:
count_alphabetic_and_emojis(s)/len(s)

0.05587989991659716

In [364]:
import json
from pathlib import Path
import re 
import emoji

from datetime import datetime

def count_alphabetic(input_string):
    count = 0
    
    for char in input_string:
        if char.isalpha():
            count += 1
    return count

def count_emojis(input_string):
    count = 0
    
    for char in input_string:
        if emoji.EMOJI_DATA:
            count += 1
    return count

def check_message(message, MAX_TEXT_LEN, symbols_treshold = 0.5):
    #remove forwarded messa
    if message.get('forwarded_from', None):
        return False
    
    if message.get('text', None):
        text = message.get('text', None)
        if len(text) > MAX_TEXT_LEN:
            return False 
        if re.search("(?P<url>https?://[^\s]+)", str(text)): # remove links 
            return False   
        alph_num = count_alphabetic(str(text))
        emoji_num = count_emojis(str(text))
        
        if (alph_num)/len(str(text)) < symbols_treshold:
            return False
    elif message.get('sticker_emoji', None):
        text = message.get('sticker_emoji', None)
    else:
        return False
        
    curr_date = datetime.strptime(message.get('date', None),'%Y-%m-%dT%H:%M:%S') 
    curr_sender = message.get('from_id', None)
    
    if curr_date is None or curr_sender is None:
        return False 
    
    return {'text':text, 'curr_date':curr_date, 'curr_sender':curr_sender}
    
    


def get_dialogues(dump_path, MAX_CONTEXT_LEN = 200, MAX_TEXT_LEN = 180, my_ids = ['user5982387868', 'user348898603'], prompt = ''): 
    # MAX_CONTEXT_LEN = 500, MAX_TEXT_LEN = 200,

    with open(dump_path) as messages_file:
            messages = json.load(messages_file)['messages']

            
    dialogues = []
    last_post_date = None
    last_sender = None
    last_anna_text = ''
    
    my_text = ''
    anna_text = ''
    curr_context = ''
    
    for message in messages:
        
        message = check_message(message, MAX_TEXT_LEN)
        if not message:
            continue
        else:
            curr_date = message['curr_date']
            curr_sender = message['curr_sender']
            curr_text = message['text']
        
        if last_post_date == None:
            last_post_date = curr_date
        if last_sender == None:
            last_sender = curr_sender
        
        
        if curr_sender != last_sender and curr_sender not in my_ids:
            #dialogues.append({'context':curr_context, 'instruction':anna_text, 'response':my_text})
            if len(anna_text) != 0:
                curr_context += ' Собеседник:' + anna_text + '\n'
            dialogues.append({'context':curr_context, 'response':my_text})
            if len(my_text) != 0:
                curr_context += ' Ты:' + my_text + '\n'
                
            last_anna_text = anna_text
            anna_text = ''
            my_text = ''
        
        if curr_sender in my_ids:
            my_text = my_text + str(curr_text) + ' \n'
        else:
            anna_text = anna_text + str(curr_text) + ' \n'

        
        if len(curr_context) > MAX_CONTEXT_LEN:
            curr_context = ''
        
        if (curr_date - last_post_date).seconds//3600 > 2: 
            
            last_post_date = None
            last_sender = None
            
            curr_input = ''
            curr_output = ''
            curr_context = ''  
        
        last_sender = curr_sender
        last_post_date = curr_date

        # TO-DO: catch reply_to_message_id
        
    return dialogues

In [365]:
import os
import glob

def find_json_files(folder_path, exceptions = ['']):
    json_files = []
    
    search_pattern = os.path.join(folder_path, '**', '*.json')
    
    for file_path in glob.iglob(search_pattern, recursive=True):
        if os.path.isfile(file_path) and file_path not in exceptions:
            json_files.append(file_path)
    
    return json_files

folder_path = '/home/box/digital_clone/dialogues'
#exceptions = ['/home/box/digital_clone/dialogues/tg_dialogues_with_anna_account_1.json']
exceptions = ['']

json_paths = find_json_files(folder_path, exceptions = exceptions)

if json_paths:
    print('-' * 110 )
    print('PARSED_PATHS:')
    print('-' * 110 )
    for json_file in json_paths:
        print(json_file)
else:
    print("There is no json files")

--------------------------------------------------------------------------------------------------------------
PARSED_PATHS:
--------------------------------------------------------------------------------------------------------------
/home/box/digital_clone/dialogues/m_1.json
/home/box/digital_clone/dialogues/anastasia.json
/home/box/digital_clone/dialogues/colya.json
/home/box/digital_clone/dialogues/dasha_hse.json
/home/box/digital_clone/dialogues/genyaa.json
/home/box/digital_clone/dialogues/mama.json
/home/box/digital_clone/dialogues/tg_dialogues_with_anna_account_2.json
/home/box/digital_clone/dialogues/misha.json
/home/box/digital_clone/dialogues/zloy_chelovek.json
/home/box/digital_clone/dialogues/nikita.json
/home/box/digital_clone/dialogues/alan.json
/home/box/digital_clone/dialogues/ekaterina.json
/home/box/digital_clone/dialogues/vova.json
/home/box/digital_clone/dialogues/dima.json
/home/box/digital_clone/dialogues/tg_dialogues_with_anna_account_2_2.json


In [366]:
# prompt = '''Тебя зовут Никита. Тебе 21 год. У тебя есть девушка Аня, вы встречаетесь 2 года и ты ее очень любишь. 
# Ты общаешься с Аней, продолжи диалог: '''

all_dialogues = []


for path in json_paths:
    # диалоги с разных аккаунтов тг
    dialogues = get_dialogues(path)
    
    if path == '/home/box/digital_clone/dialogues/tg_dialogues_with_anna_account_2_2.json':
        all_dialogues += dialogues[:5000]
    else:
        all_dialogues += dialogues
    print(len(dialogues))

235
353
79
2655
60
897
3475
5863
344
106
452
140
155
1214
23032


In [371]:
len(all_dialogues)

21028

In [372]:
all_dialogues

[{'context': ' Собеседник:Почему ты тут в сети был \nА в вк нет? \n\n',
  'response': 'Я случайно зашел в телегу ... \n'},
 {'context': ' Собеседник:Почему ты тут в сети был \nА в вк нет? \n\n Ты:Я случайно зашел в телегу ... \n\n Собеседник:😄 \n\n',
  'response': 'Аоаоаоаоаоа \n'},
 {'context': ' Собеседник:Почему ты тут в сети был \nА в вк нет? \n\n Ты:Я случайно зашел в телегу ... \n\n Собеседник:😄 \n\n Ты:Аоаоаоаоаоа \n\n Собеседник:😴 \n\n',
  'response': 'На какой акк ? \n'},
 {'context': ' Собеседник:Почему ты тут в сети был \nА в вк нет? \n\n Ты:Я случайно зашел в телегу ... \n\n Собеседник:😄 \n\n Ты:Аоаоаоаоаоа \n\n Собеседник:😴 \n\n Ты:На какой акк ? \n\n Собеседник:На любой \nАхахх \n\n',
  'response': 'Ок \n'},
 {'context': ' Собеседник:Почему ты тут в сети был \nА в вк нет? \n\n Ты:Я случайно зашел в телегу ... \n\n Собеседник:😄 \n\n Ты:Аоаоаоаоаоа \n\n Собеседник:😴 \n\n Ты:На какой акк ? \n\n Собеседник:На любой \nАхахх \n\n Ты:Ок \n\n Собеседник:У тебя же их два? \nНе три

In [374]:
with open('./all_dialogues.json', 'w', encoding = 'utf8') as json_file:
    json.dump(all_dialogues, json_file, indent=4, ensure_ascii=False)